<a href="https://colab.research.google.com/github/clionelove123/temp_test/blob/main/Chap_5_Artificial_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
# 텐서플로우를 이용한 ANN(Artificial Neural Networks) 구현

import tensorflow as tf

In [ ]:
# MNIST 데이터를 다운로드 합니다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# 이미지들을 float32 데이터 타입으로 변경합니다.
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
# 28*28 형태의 이미지를 784차원으로 flattening 합니다.
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
# [0, 255] 사이의 값을 [0, 1]사이의 값으로 Normalize합니다.
x_train, x_test = x_train / 255., x_test / 255.
# 레이블 데이터에 one-hot encoding을 적용합니다.
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

In [ ]:
# 학습을 위한 설정값들을 정의합니다.
learning_rate = 0.001
num_epochs = 50     # 학습횟수
batch_size = 256    # 배치개수
display_step = 1    # 손실함수 출력 주기
input_size = 784    # 28 * 28
hidden1_size = 256
hidden2_size = 256
output_size = 10

In [ ]:
# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size)

# ANN 모델을 정의합니다.
class ANN(object):
  # ANN 모델을 위한 tf.Variable들을 정의합니다.
  def __init__(self):
    self.W1 = tf.Variable(tf.random.normal(shape=[input_size, hidden1_size]))
    self.b1 = tf.Variable(tf.random.normal(shape=[hidden1_size]))
    self.W2 = tf.Variable(tf.random.normal(shape=[hidden1_size, hidden2_size]))
    self.b2 = tf.Variable(tf.random.normal(shape=[hidden2_size]))
    self.W_output = tf.Variable(tf.random.normal(shape=[hidden2_size, output_size]))
    self.b_output = tf.Variable(tf.random.normal(shape=[output_size]))

  def __call__(self, x):
    H1_output = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
    H2_output = tf.nn.relu(tf.matmul(H1_output, self.W2) + self.b2)
    logits = tf.matmul(H2_output, self.W_output) + self.b_output

    return logits

In [ ]:
# cross-entropy 손실 함수를 정의합니다.
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# 최적화를 위한 Adam 옵티마이저를 정의합니다.
optimizer = tf.optimizers.Adam(learning_rate)

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, vars(model).values())
  optimizer.apply_gradients(zip(gradients, vars(model).values()))

# 모델의 정확도를 출력하는 함수를 정의합니다.
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

In [ ]:
# ANN 모델을 선언합니다.
ANN_model = ANN()

# 지정된 횟수만큼 최적화를 수행합니다.
for epoch in range(num_epochs):
  average_loss = 0.
  total_batch = int(x_train.shape[0] / batch_size)
  # 모든 배치들에 대해서 최적화를 수행합니다.
  for batch_x, batch_y in train_data:
    # 옵티마이저를 실행해서 파라마터들을 업데이트합니다.
    _, current_loss = train_step(ANN_model, batch_x, batch_y), cross_entropy_loss(ANN_model(batch_x), batch_y)
    # 평균 손실을 측정합니다.
    average_loss += current_loss / total_batch
  # 지정된 epoch마다 학습결과를 출력합니다.
  if epoch % display_step == 0:
    print("반복(Epoch): %d" % (epoch+1), "정확도 (Accuracy): %f" % compute_accuracy(ANN_model(batch_x), batch_y), "손실 함수(Loss): %f" % average_loss)
    print("Test데이터 - 정확도(Accuracy): %f" % compute_accuracy(ANN_model(x_test), y_test)," / 손실(loss): %f" % cross_entropy_loss(ANN_model(x_test), y_test)) # 정확도: 약 94%


반복(Epoch): 1 정확도 (Accuracy): 0.760417 손실 함수(Loss): 281.867645
Test데이터 - 정확도(Accuracy): 0.798000  / 손실(loss): 82.672173
반복(Epoch): 2 정확도 (Accuracy): 0.802083 손실 함수(Loss): 60.784351
Test데이터 - 정확도(Accuracy): 0.859900  / 손실(loss): 50.925350
반복(Epoch): 3 정확도 (Accuracy): 0.885417 손실 함수(Loss): 38.882629
Test데이터 - 정확도(Accuracy): 0.882400  / 손실(loss): 39.262642
반복(Epoch): 4 정확도 (Accuracy): 0.885417 손실 함수(Loss): 28.550945
Test데이터 - 정확도(Accuracy): 0.899500  / 손실(loss): 32.599354
반복(Epoch): 5 정확도 (Accuracy): 0.916667 손실 함수(Loss): 22.191895
Test데이터 - 정확도(Accuracy): 0.906600  / 손실(loss): 28.604021
반복(Epoch): 6 정확도 (Accuracy): 0.916667 손실 함수(Loss): 17.753979
Test데이터 - 정확도(Accuracy): 0.913600  / 손실(loss): 25.543058
반복(Epoch): 7 정확도 (Accuracy): 0.968750 손실 함수(Loss): 14.406864
Test데이터 - 정확도(Accuracy): 0.920000  / 손실(loss): 23.476322
반복(Epoch): 8 정확도 (Accuracy): 0.937500 손실 함수(Loss): 11.705971
Test데이터 - 정확도(Accuracy): 0.920300  / 손실(loss): 21.844490
반복(Epoch): 9 정확도 (Accuracy): 0.958333 손실 함수(Loss): 9.65

In [ ]:
# 테스트 데이터를 이용해서 학습된 모델이 얼마나 정확한지 정확도를 출력합니다.
print("정확도(Accuracy): %f" % compute_accuracy(ANN_model(x_test), y_test)) # 정확도: 약 94%

정확도(Accuracy): 0.951800
